In [1]:
#Import our libraries
import time
import requests
import json
import xml.etree.ElementTree as ET  # for parsing XML
import xmltodict
import pandas as pd
import csv
import base64 
import sqlalchemy

#Import our helper functions
import helper_functions_labor_distribution

# Step 1: Get report path

In [2]:
#report_path = report_lst[report_lst['ReportName'] == 'Organizational Levels']['ReportPath'].loc[57]
report_path = helper_functions_labor_distribution.get_report_file_path('Organizational Levels')

report_path 

"/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/folder[@name='Export to SQL']/report[@name='Organizational Levels']"

## Step 2: Retrieve the report and export as a CSV

In [3]:
df = helper_functions_labor_distribution.retrieve_report(report_path) 
df 

,gl_segment,level,"manager_(last_suffix,_first_mi)",manager's_alternate_email,manager's_email,organization_code,organization,status_code,status
0,ENT,1,Lenart_ Sylvia B.,'@xs:nil': 'true',S.Lenart@proliancesurgeons.com,007010,PEENT - CFED,A,Active
1,ENT,1,Lenart_ Sylvia B.,'@xs:nil': 'true',S.Lenart@proliancesurgeons.com,007019,PEENT - CFED Optical,A,Active
2,EVS,1,Wilson_ Carmen M.,'@xs:nil': 'true',C.Wilson@proliancesurgeons.com,005100,PESC - ASC,A,Active
3,MNJ,1,Perdue_ Debra L.,'@xs:nil': 'true',D.Perdue@proliancesurgeons.com,003922,MJSS - Podiatry Bellevue,I,Inactive
4,MNJ,1,Perdue_ Debra L.,'@xs:nil': 'true',D.Perdue@proliancesurgeons.com,003923,MJSS - Podiatry Issaquah,I,Inactive
...,...,...,...,...,...,...,...,...,...
3258,OCW,1,'@xs:nil': 'true','@xs:nil': 'true','@xs:nil': 'true',001902,OPA - MRI,A,Active
3259,STE,1,'@xs:nil': 'true','@xs:nil': 'true','@xs:nil': 'true',002500,EDOC,A,Active
3260,STE,1,'@xs:nil': 'true','@xs:nil': 'true','@xs:nil': 'true',002501,EDOC - COSJS,A,Active
3261,CPT,1,'@xs:nil': 'true','@xs:nil': 'true','@xs:nil': 'true',006900,COHT,A,Active


In [4]:
df.columns

Index(['gl_segment', 'level', 'manager_(last_suffix,_first_mi)',
       'manager's_alternate_email', 'manager's_email', 'organization_code',
       'organization', 'status_code', 'status'],
      dtype='object')

In [5]:
#Remove the null values so that they say 0 instead
df['level'] = df['level'].astype(str).replace("'@xs:nil': 'true'", '0')
df['organization_code'] = df['organization_code'].astype(str).replace("'@xs:nil': 'true'", '0')

# Step 3: Export the report as a CSV

In [6]:
df.to_csv('ukg_bulk_insert.csv', index=False)

# Step 4: Insert the report into BiData2 as a table

In [7]:
helper_functions_labor_distribution.create_sql_table_v2('ZZ_TEST_OrgLevel')

# Step 5: Close the API connection

In [8]:
helper_functions_labor_distribution.close_api_connection()

'<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/LogOffResponse</a:Action></s:Header><s:Body><LogOffResponse xmlns="http://www.ultipro.com/dataservices/bidata/2"><LogOffResult xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><ServiceId i:nil="true"/><ClientAccessKey>PLAB4</ClientAccessKey><Token i:nil="true"/><Status>LoggedOff</Status><StatusMessage i:nil="true"/><InstanceKey i:nil="true"/></LogOffResult></LogOffResponse></s:Body></s:Envelope>'

The code to create the SQL table:
CREATE TABLE ZZ_TEST_Organizational_Levels(
	gl_segement VARCHAR(255),
	level_ VARCHAR(255),
	manager_name VARCHAR(255),
	manager_alternate_email VARCHAR(255),
	manager_email VARCHAR(255),
	organization_code VARCHAR(255),
	organization VARCHAR(255),
	status_code VARCHAR(255),
	status_ VARCHAR(255))